<a href="https://colab.research.google.com/github/Khairul-islam99/Multimodal-LLM-Fine-Tuning/blob/main/Multimodal_LLM_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets -q

from datasets import load_dataset

# "gsm8k" ডেটাসেটের "main" সাবসেটটি লোড করুন (এতে train স্প্লিট আছে)
dataset = load_dataset("gsm8k", "main", split="train")

# প্রথম স্যাম্পলটি দেখুন
print("--- gsm8k ডেটাসেটের গঠন ---")
print(dataset[0])

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

--- gsm8k ডেটাসেটের গঠন ---
{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}


In [ ]:
def convert_math_conversation(sample):
    # উত্তরের মধ্যে ধাপে ধাপে সমাধান এবং ফাইনাল উত্তর দুটোই আছে
    solution_text = sample["answer"]

    conversation = [
        { "role": "user",
          "content" : [
            # gsm8k একটি টেক্সট-ভিত্তিক ডেটাসেট, তাই এখানে কোনো ছবি নেই
            {"type" : "text", "text" : sample["question"]}
          ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text", "text" : solution_text}
          ]
        },
    ]
    return { "messages" : conversation }

# --- পুরো ডেটাসেটকে রূপান্তর করুন ---
converted_math_dataset = [convert_math_conversation(sample) for sample in dataset]

# প্রথম রূপান্তরিত স্যাম্পলটি দেখুন
print("\n--- রূপান্তরিত চ্যাট ফরম্যাট (প্রথম স্যাম্পল) ---")
print(converted_math_dataset[0])


--- রূপান্তরিত চ্যাট ফরম্যাট (প্রথম স্যাম্পল) ---
{'messages': [{'role': 'user', 'content': [{'type': 'text', 'text': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}]}]}


In [ ]:
import json

filename = "converted_math_dataset.jsonl"

with open(filename, 'w') as f:
    for item in converted_math_dataset:
        f.write(json.dumps(item) + '\n')

print(f"✅ ডেটাসেট '{filename}'-এ সেভ করা হয়েছে।")

✅ ডেটাসেট 'converted_math_dataset.jsonl'-এ সেভ করা হয়েছে।


In [ ]:
%%capture
!pip install gradio
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    # datasets লাইব্রেরিটি ইনস্টল করুন
    !pip install datasets
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
# আপনার নোটবুকের সাথে মিল রেখে transformers এবং trl এর ভার্সন পিন করা হলো
!pip install transformers==4.57.0
!pip install --no-deps trl==0.22.2

In [ ]:
from unsloth import FastVisionModel
import torch

# আপনার বেস মডেলটি লোড করুন
model, tokenizer = FastVisionModel.from_pretrained(
    "khairul5/BINI-Qwen3-VL-MyMirror",
    load_in_4bit = True, # ৪-বিটে লোড করা হচ্ছে
    use_gradient_checkpointing = "unsloth",
)

# মডেলের ওপর LoRA অ্যাডাপ্টার যোগ করুন (PEFT)
# এই কোডটি মডেলকে ট্রেইন করার জন্য প্রস্তুত করবে
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r = 16,           # LoRA Rank
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

print("\n✅ মডেল এবং LoRA অ্যাডাপ্টার সফলভাবে লোড হয়েছে।")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.2: Fast Qwen3_Vl patching. Transformers: 4.57.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]


✅ মডেল এবং LoRA অ্যাডাপ্টার সফলভাবে লোড হয়েছে।


In [ ]:
from google.colab import files
from datasets import load_dataset, Dataset
import json

# --- ১. ট্রেইন ডেটাসেট আপলোড ও লোড করুন ---
print("দয়া করে আপনার 'converted_math_dataset.jsonl' ফাইলটি আপলোড করুন:")
uploaded = files.upload()

if len(uploaded.keys()) == 0:
    raise ValueError("❌ কোনো ফাইল আপলোড করা হয়নি। দয়া করে সেলটি আবার রান করুন।")

jsonl_filename = list(uploaded.keys())[0]
train_dataset = load_dataset('json', data_files=jsonl_filename, split='train')
print(f"✅ ট্রেইন ডেটাসেট লোড হয়েছে: {train_dataset}")


# --- ২. ইভালুয়েশন (টেস্ট) ডেটাসেট লোড ও প্রসেস করুন ---
print("\n'gsm8k' থেকে টেস্ট ডেটাসেট লোড করা হচ্ছে...")
eval_dataset_raw = load_dataset("gsm8k", "main", split="test")

# একই ফরম্যাটিং ফাংশনটি আবার ডিফাইন করুন
def convert_math_conversation(sample):
    solution_text = sample["answer"]
    conversation = [
        {"role": "user", "content": [{"type": "text", "text": sample["question"]}]},
        {"role": "assistant", "content": [{"type": "text", "text": solution_text}]},
    ]
    return {"messages": conversation}

# টেস্ট ডেটাসেটকে প্রসেস করুন
converted_eval_dataset = [convert_math_conversation(sample) for sample in eval_dataset_raw]
eval_dataset = Dataset.from_list(converted_eval_dataset)

print(f"✅ ইভালুয়েশন ডেটাসেট প্রস্তুত হয়েছে: {eval_dataset}")

দয়া করে আপনার 'converted_math_dataset.jsonl' ফাইলটি আপলোড করুন:


Saving converted_math_dataset.jsonl to converted_math_dataset (1).jsonl


Generating train split: 0 examples [00:00, ? examples/s]

✅ ট্রেইন ডেটাসেট লোড হয়েছে: Dataset({
    features: ['messages'],
    num_rows: 7473
})

'gsm8k' থেকে টেস্ট ডেটাসেট লোড করা হচ্ছে...


README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

✅ ইভালুয়েশন ডেটাসেট প্রস্তুত হয়েছে: Dataset({
    features: ['messages'],
    num_rows: 1319
})


In [ ]:
from trl import SFTTrainer, SFTConfig # ❗️(নতুন) SFTConfig ইম্পোর্ট করুন
from unsloth.trainer import UnslothVisionDataCollator

# Qwen3-VL মডেলের জন্য UnslothVisionDataCollator ব্যবহার করা আবশ্যক
data_collator = UnslothVisionDataCollator(model, tokenizer)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    # eval_dataset = eval_dataset, # ❗️(বাদ দেওয়া হলো) আমরা এটি পরে ব্যবহার করব
    data_collator = data_collator,
    args = SFTConfig( # ❗️(নতুন) TrainingArguments-এর বদলে SFTConfig
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1, # পুরো ডেটাসেটকে ১ বার ট্রেইন করুন

        learning_rate = 2e-4,
        optim = "adamw_8bit",

        logging_steps = 10, # প্রতি ১০ স্টেপে ট্রেইন লস দেখান

        output_dir = "outputs",
        warmup_steps = 5,
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",

        # --- Vision মডেলের জন্য আবশ্যক প্যারামিটার ---
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_length = 2048,
    ),
)

Unsloth: Model does not have a default image size - using 512


In [ ]:
print("🚀 ফাইন-টিউনিং শুরু হচ্ছে...")
# ট্রেইনিং শুরু করুন
trainer_stats = trainer.train()
print("✅ ফাইন-টিউনিং সম্পন্ন হয়েছে!")

# ট্রেইনিংয়ের সময় মেমরি ব্যবহারের পরিসংখ্যান দেখুন
gpu_stats = torch.cuda.get_device_properties(0)
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"Peak reserved memory = {used_memory} GB.")

The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 ফাইন-টিউনিং শুরু হচ্ছে...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 935
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 51,346,944 of 8,818,470,640 (0.58% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,0.326100
20,0.152000
30,0.123300
40,0.102100
50,0.099200
60,0.100500
70,0.099600
80,0.092400
90,0.093700
100,0.095400


✅ ফাইন-টিউনিং সম্পন্ন হয়েছে!
GPU = Tesla T4. Max memory = 14.741 GB.
Peak reserved memory = 7.387 GB.


In [ ]:
print("ম্যানুয়াল ইভালুয়েশন শুরু হচ্ছে...")
# trainer.evaluate() কমান্ডটি eval_dataset-এর ওপর চলবে
eval_results = trainer.evaluate(eval_dataset)

print("\n--- ইভালুয়েশন ফলাফল ---")
print(f"Eval Loss: {eval_results['eval_loss']}")

Unsloth: Not an error, but Qwen3VLForConditionalGeneration does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


ম্যানুয়াল ইভালুয়েশন শুরু হচ্ছে...



--- ইভালুয়েশন ফলাফল ---
Eval Loss: 0.8971353769302368


In [ ]:
from google.colab import files
import shutil

# ১. অ্যাডাপ্টারগুলো Colab-এ সেভ করুন
model.save_pretrained("bini_math_model_adapters")
tokenizer.save_pretrained("bini_math_model_adapters")

print("✅ অ্যাডাপ্টার সেভ হয়েছে। এখন জিপ করা হচ্ছে...")

# ২. ফোল্ডারটিকে জিপ করুন
shutil.make_archive("bini_math_model_adapters", 'zip', "bini_math_model_adapters")

# ৩. জিপ ফাইলটি ডাউনলোড করুন
files.download("bini_math_model_adapters.zip")

✅ অ্যাডাপ্টার সেভ হয়েছে। এখন জিপ করা হচ্ছে...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install gradio -q

In [ ]:
# আপনার বিদ্যমান 'model' অবজেক্টকে ইনফারেন্স মোডে সেট করুন
FastVisionModel.for_inference(model)
print("✅ আপনার ফাইন-টিউনড মডেল ইনফারেন্সের জন্য প্রস্তুত।")

✅ আপনার ফাইন-টিউনড মডেল ইনফারেন্সের জন্য প্রস্তুত।


In [ ]:
import gradio as gr
from PIL import Image
import io
from threading import Thread
from transformers import TextIteratorStreamer

def format_chat_history(chat_history):
    """
    Gradio-এর চ্যাট হিস্ট্রিকে মডেলের ফরম্যাটে রূপান্তর করে।
    """
    messages = []
    if chat_history is None:
        return messages

    for turn in chat_history:
        user_msg, assistant_msg = turn
        messages.append({"role": "user", "content": [{"type": "text", "text": user_msg}]})
        if assistant_msg:
             messages.append({"role": "assistant", "content": [{"type": "text", "text": assistant_msg}]})

    return messages

def chat_stream(image_input, text_input, chat_history):
    """
    মডেলের রেসপন্স স্ট্রিম করার জন্য জেনারেটর ফাংশন।
    """

    # ১. হিস্ট্রি এবং নতুন প্রম্পট ফরম্যাট করুন
    messages = format_chat_history(chat_history)

    # ২. নতুন ইউজার প্রম্পট প্রস্তুত করুন
    new_user_content = []
    pil_image = None

    if image_input is not None:
        pil_image = image_input
        new_user_content.append({"type": "image"})

    if text_input:
        new_user_content.append({"type": "text", "text": text_input})
    else:
        if image_input is not None:
             new_user_content.append({"type": "text", "text": "Describe this image."})
             text_input = "Describe this image."
        else:
            yield chat_history
            return

    messages.append({"role": "user", "content": new_user_content})

    # ৩. চ্যাট টেমপ্লেট অ্যাপ্লাই করুন
    input_text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True
    )

    # ৪. মডেলের জন্য ইনপুট প্রস্তুত করুন
    inputs = tokenizer(
        pil_image,
        input_text,
        add_special_tokens = False,
        return_tensors = "pt",
    ).to("cuda")

    # ৫. স্ট্রিমার সেটআপ করুন
    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_kwargs = dict(
        inputs,
        streamer=streamer,
        max_new_tokens=1024,
        use_cache=True,
        temperature=1.0,
        min_p=0.1
    )

    # ৬. আলাদা থ্রেডে জেনারেশন শুরু করুন
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    # ৭. Gradio UI-তে নতুন টোকেন পাঠান
    if chat_history is None:
        chat_history = []

    chat_history.append((text_input, ""))

    response_text = ""
    for new_token in streamer:
        response_text += new_token
        chat_history[-1] = (text_input, response_text)
        yield chat_history

def clear_chat():
    """চ্যাট হিস্ট্রি এবং ইনপুট ক্লিয়ার করে।"""
    return [], None, ""

# --- Gradio ইন্টারফেস তৈরি ---

theme = gr.themes.Soft(
    primary_hue="blue",
    secondary_hue="orange",
    font=gr.themes.GoogleFont("Open Sans")
)

with gr.Blocks(theme=theme, title="Bini (Math) Chat") as demo:
    with gr.Column():
        # --- হেডার (পরিবর্তিত) ---
        gr.Markdown(
            """
            <div style="text-align: center;">
                <h1>🤖 Bini (Math Finetuned)</h1>
                <p>Your personal multimodal assistant. Now with improved Math reasoning!</p>
            </div>
            """
        )

        # --- চ্যাট উইন্ডো (পরিবর্তিত) ---
        chatbot = gr.Chatbot(
            value=[[None, "Hello! I'm Bini. I've been finetuned on math problems. How can I help?"]],
            label="Chat",
            height=500,
            bubble_full_width=False,
        )

        # --- ইনপুট সেকশন ---
        with gr.Row(equal_height=False):
            with gr.Column(scale=1, min_width=200):
                image_box = gr.Image(
                    type="pil",
                    label="Upload Image (Optional)",
                    sources=["upload"],
                    height=150
                )
            with gr.Column(scale=4):
                text_box = gr.Textbox(
                    label="Send a message",
                    placeholder="Type your math problem or ask about an image...",
                    show_label=False,
                    lines=3
                )

        # --- বাটন ---
        with gr.Row():
            clear_btn = gr.Button("Clear Chat", variant="stop")
            send_btn = gr.Button("Send", variant="primary")

        # --- উদাহরণ (পরিবর্তিত) ---
        gr.Examples(
            examples=[
                ["Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"],
                ["What is in this image?"],
                ["Hello, who are you?"],
            ],
            inputs=[text_box],
            label="Example Prompts (Try the math one!)"
        )

        # --- ফুটার (পরিবর্তিত) ---
        gr.Markdown(
            """
            <div style="text-align: center; font-size: small; color: grey;">
                <p><b>Bini</b> was built by <b>Md Khairrul Islam</b>.<br>
                (Powered by unsloth/Qwen3-VL-8B-Instruct + Math Finetune)</p>
            </div>
            """
        )

    # --- ইভেন্ট হ্যান্ডলার ---

    send_btn.click(
        chat_stream,
        inputs=[image_box, text_box, chatbot],
        outputs=[chatbot],
    ).then(
        lambda: (None, ""),
        outputs=[image_box, text_box]
    )

    text_box.submit(
        chat_stream,
        inputs=[image_box, text_box, chatbot],
        outputs=[chatbot],
    ).then(
        lambda: (None, ""),
        outputs=[image_box, text_box]
    )

    clear_btn.click(
        clear_chat,
        outputs=[chatbot, image_box, text_box]
    )

# অ্যাপটি চালু করুন
demo.launch(debug=True, share=True)

/tmp/ipython-input-3842669214.py:118: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-3842669214.py:118: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://258c7d85963614271b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
